In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Boolean
from sqlalchemy.orm import declarative_base, sessionmaker

Base = declarative_base()

# Define a model for the table
class UserInput(Base):
    __tablename__ = 'user_input'
    
    id = Column(Integer, primary_key=True)
    selected_date = Column(String, unique=True)  # Ensure date is unique
    broth = Column(Boolean)
    kavas = Column(Boolean)
    submitted = Column(Boolean, default=False)

# Create an SQLite database and the table
engine = create_engine('sqlite:///bayesian_life.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)



In [3]:
import streamlit as st
from datetime import date


class BayesianLifeApp:

    def __init__(self):
        st.title("Bayesian Life")

        # Collect inputs from user
        selected_date = st.text_input("Select Date (YYYY-MM-DD)", value=date.today().strftime("%Y-%m-%d"))
        broth = st.checkbox("Broth")
        kavas = st.checkbox("Kavas")
        resubmit = st.checkbox("Resubmit")

        if st.button("Submit"):
            result = self.bayesian_life(selected_date, broth, kavas, resubmit)
            st.text(result)

    def bayesian_life(self, selected_date, broth, kavas, resubmit=False):
        session = Session()

        # Check if there's already an entry for the given date
        existing_entry = session.query(UserInput).filter_by(selected_date=selected_date).first()

        if existing_entry and existing_entry.submitted and not resubmit:
            session.close()
            return "Already submitted for this date. Would you like to resubmit?"

        if existing_entry and resubmit:
            # Update the existing entry
            existing_entry.broth = broth
            existing_entry.kavas = kavas
            existing_entry.submitted = True
        else:
            # Insert a new entry
            new_input = UserInput(selected_date=selected_date, broth=broth, kavas=kavas, submitted=True)
            session.add(new_input)

        session.commit()
        session.close()

        return "Your data has been submitted successfully!"

# Create an instance of the app and run it
if __name__ == "__main__":
    app = BayesianLifeApp()


2023-10-19 16:52:23.047 
  command:

    streamlit run c:\Users\Adaptation\.conda\envs\BayesianLife\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [4]:
!streamlit run c:\Users\Adaptation\.conda\envs\BayesianLife\Lib\site-packages\ipykernel_launcher.py


^C


In [2]:
import gradio as gr
from gradio.components import Textbox, Checkbox
from datetime import date

class BayesianLifeApp:

    def __init__(self):
        # Define the function that the interface will call
        def bayesian_life(selected_date, broth, kavas, resubmit=False):
            session = Session()

            # Check if there's already an entry for the given date
            existing_entry = session.query(UserInput).filter_by(selected_date=selected_date).first()

            if existing_entry and existing_entry.submitted and not resubmit:
                session.close()
                return "Already submitted for this date. Would you like to resubmit?"

            if existing_entry and resubmit:
                # Update the existing entry
                existing_entry.broth = broth
                existing_entry.kavas = kavas
                existing_entry.submitted = True
            else:
                # Insert a new entry
                new_input = UserInput(selected_date=selected_date, broth=broth, kavas=kavas, submitted=True)
                session.add(new_input)

            session.commit()
            session.close()
            
            return "Your data has been submitted successfully!"

        # Define the Gradio interface
        self.interface = gr.Interface(
            fn=bayesian_life,
            inputs=[
                Textbox(label="Select Date (YYYY-MM-DD)", value=date.today().strftime("%Y-%m-%d")),
                Checkbox(label="Broth"),
                Checkbox(label="Kavas"),
                Checkbox(label="Resubmit", value=False)
            ],
            outputs="text",
            title="Bayesian Life"
        )

    def run(self):
        # Launch the Gradio app
        self.interface.launch(server_name='192.168.0.51')

# Create an instance of the app and run it
app = BayesianLifeApp()
app.run()


c:\Users\Adaptation\.conda\envs\BayesianLife\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Adaptation\AppData\Local\Temp\ipykernel_18036\3173902626.py:33: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Textbox(label="Select Date (YYYY-MM-DD)", default=date.today().strftime("%Y-%m-%d")),
C:\Users\Adaptation\AppData\Local\Temp\ipykernel_18036\3173902626.py:33: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(label="Select Date (YYYY-MM-DD)", default=date.today().strftime("%Y-%m-%d")),
C:\Users\Adaptation\AppData\Local\Temp\ipykernel_18036\3173902626.py:33: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no ef

Running on local URL:  http://192.168.0.51:7860

To create a public link, set `share=True` in `launch()`.


In [1]:
import hashlib

password = "Wog19881992Rp:"
hashed_pw = hashlib.sha256(password.encode()).hexdigest()
print(hashed_pw)

14fb8077481fa48aa86d2cb7c2f948eae410dda5a322ef7d38f5911efd5a6707


In [4]:
import gradio as gr
import pandas as pd
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

class BayesianLifeDataApp:

    def __init__(self):
        
        def fetch_and_save(start_date=None, end_date=None, fetch_all=False, save_csv=False, csv_filename="bayesian_life_data.csv"):
            session = Session()

            # Get data from the database
            query = session.query(UserInput)
            
            if not fetch_all:
                if start_date:
                    query = query.filter(UserInput.selected_date >= start_date)
                if end_date:
                    query = query.filter(UserInput.selected_date <= end_date)

            df = pd.read_sql(query.statement, query.session.bind)
            
            session.close()

            # Save dataframe as CSV if required
            if save_csv:
                df.to_csv(csv_filename, index=False)

            return f"Data fetched successfully! Saved to {csv_filename}", df

        # Define the Gradio interface
        self.interface = gr.Interface(
            fn=fetch_and_save,
            inputs=[
                gr.Textbox(label="Start Date (YYYY-MM-DD)", optional=True),
                gr.Textbox(label="End Date (YYYY-MM-DD)", optional=True),
                gr.Checkbox(label="Fetch All Data"),
                gr.Checkbox(label="Save as CSV"),
                gr.Textbox(label="CSV Filename", default="bayesian_life_data.csv")
            ],
            outputs=[
                "text",
                gr.Dataframe(type="pandas")
            ],
            title="Bayesian Life Data"
        )

    def run(self):
        # Launch the Gradio app
        self.interface.launch()

# Create the SQLite database connection
engine = create_engine('sqlite:///bayesian_life.db')
Session = sessionmaker(bind=engine)

# Create an instance of the app and run it
app = BayesianLifeDataApp()
app.run()


c:\Users\Adaptation\.conda\envs\BayesianLife\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Adaptation\AppData\Local\Temp\ipykernel_26224\753551001.py:36: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.Textbox(label="Start Date (YYYY-MM-DD)", optional=True),
C:\Users\Adaptation\AppData\Local\Temp\ipykernel_26224\753551001.py:37: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.Textbox(label="End Date (YYYY-MM-DD)", optional=True),
C:\Users\Adaptation\AppData\Local\Temp\ipykernel_26224\753551001.py:40: GradioUnusedKwargWarning: You have unused kwarg parameters in Textbox, please remove them: {'default': 'bayesian_life_data.csv'}
  gr.Textbox(label="CSV Filename", default="bayesian_life_data.csv")


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\Adaptation\.conda\envs\BayesianLife\Lib\site-packages\gradio\routes.py", line 534, in predict
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Adaptation\.conda\envs\BayesianLife\Lib\site-packages\gradio\route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Adaptation\.conda\envs\BayesianLife\Lib\site-packages\gradio\blocks.py", line 1550, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Adaptation\.conda\envs\BayesianLife\Lib\site-packages\gradio\blocks.py", line 1185, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Adaptation\.conda\envs\BayesianLife\Lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return

In [ ]:
dir(gr.inputs)


In [3]:
import streamlit_authenticator as stauth

hashed_passwords = stauth.Hasher(['Wog19881992Rp:']).generate()
hashed_passwords

['$2b$12$W/ECJdQ88TSSx0yVdGL/B.7Bv2j8zBs.RKH/yyq4YZuIjivZuY13u']